SO 
Here is how I'm gonna do this
as a first try, without any preprocessing, simply combine all the data into a single
dataframe, regardless of threshold, just to nail down how the data needs to be structured

one by one, read in the csv files, creating one line in the data frame for every 30 frames
then write the identity in terms of video, bramble-or-rose, startframe, id, extracted body-length, as the 
first 5 elements in the row. The 6th entry is then the size class, so we can use that as a
label. without any smoothing, angles, or speeds, the shape of the data-frame will then be

(n, 30 * num_key_points + 6)

to run dimensionality reduction on that data, simply repeat the setup above:

In [1]:
import pickle
import os
import pandas as pd
from umap import UMAP
from sklearn.manifold import TSNE
import plotly.express as px
from os import listdir
from os.path import isfile, join

In [10]:
input_folder = "I:/WOLO/WOLO/EAEAAO/EXTRACTED_POSE_FEATURE_VECTORS"
df_ant_poses_list =  [os.path.join(input_folder,f) for f in os.listdir(input_folder) if os.path.isfile(os.path.join(input_folder, f))]

df_ant_poses_dfs = [pd.read_pickle(filename) for filename in df_ant_poses_list]

TypeError: cannot concatenate object of type '<class 'str'>'; only Series and DataFrame objs are valid

In [17]:
# stack the DataFrames
df_ant_poses = pd.concat(df_ant_poses_dfs, ignore_index=True, axis=0)

features = df_ant_poses.loc[:, 'raw_pose_0':] # use all entries and exclude labels
labels =  df_ant_poses.loc[:, :"size_class"]

norm_features = (features-features.min())/(features.max()-features.min())

In [18]:
print("extracted feature vector contains",norm_features.shape[0],"instances and", norm_features.shape[1], "features")
labels

extracted feature vector contains 1315360 instances and 2891 features


,video,material,startframe,id,speed,size_class
0,2019-07-22_bramble_left,bramble,80639,5727,7.615634,1
1,2019-07-22_bramble_left,bramble,80664,5727,9.038922,1
2,2019-07-22_bramble_left,bramble,80689,5727,12.883899,1
3,2019-07-22_bramble_left,bramble,80714,5727,12.919391,0
4,2019-07-22_bramble_left,bramble,80739,5727,9.704732,0
...,...,...,...,...,...,...
1315355,2019-08-22_rose_left,rose,74412,1101,21.059703,1
1315356,2019-08-22_rose_left,rose,74437,1101,19.766818,1
1315357,2019-08-22_rose_left,rose,74462,1101,15.489649,1
1315358,2019-08-22_rose_left,rose,74487,1101,16.824436,1


In [ ]:
# as a test, run a 2D tSNE
tsne = TSNE(n_components=2, random_state=0, init="pca", learning_rate="auto", perplexity=50)
projections = tsne.fit_transform(norm_features)

In [ ]:
# now take the produced embedding and plot with different colour overlays to group by class, speed, or material
fig_size = px.scatter(
    projections, x=0, y=1,
    color=df_ant_poses.size_class, labels={'color': 'size_class'},
    width=600, height=500) 

fig_size.write_image("tSNE_fig_size.svg")

fig_speed = px.scatter(
    projections, x=0, y=1,
    color=df_ant_poses.speed, labels={'color': 'speed'},
    width=600, height=500) 

fig_speed.write_image("tSNE_fig_speed.svg")

fig_material = px.scatter(
    projections, x=0, y=1,
    color=df_ant_poses.material, labels={'color': 'material'},
    width=600, height=500) 

fig_material.write_image("tSNE_fig_material.svg")

fig_size.show()
fig_speed.show()
fig_material.show()

In [ ]:
umap_2d = UMAP(n_components=2, init='random')#, random_state=0)
umap_3d = UMAP(n_components=3, init='random')#, random_state=0)

proj_2d = umap_2d.fit_transform(norm_features)
proj_3d = umap_3d.fit_transform(norm_features)

fig_size_2d = px.scatter(
    proj_2d, x=0, y=1,
    color=df_ant_poses.size_class, labels={'color': 'size_class'},
    width=600, height=500
)

fig_size_3d = px.scatter_3d(
    proj_3d, x=0, y=1, z=2,
    color=df_ant_poses.size_class, labels={'color': 'size_class'},
    width=600, height=500
)
fig_size_3d.update_traces(marker_size=5)


fig_speed_2d = px.scatter(
    proj_2d, x=0, y=1,
    color=df_ant_poses.speed, labels={'color': 'speed'},
    width=600, height=500
)

fig_speed_3d = px.scatter_3d(
    proj_3d, x=0, y=1, z=2,
    color=df_ant_poses.speed, labels={'color': 'speed'},
    width=600, height=500
)
fig_speed_3d.update_traces(marker_size=5)

fig_size_2d.show()
fig_size_3d.show()

fig_speed_2d.show()
fig_speed_3d.show()